In [10]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

import pandas as pd

from joblib import Parallel, delayed

In [11]:
scoremaps_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremaps_lossless'

In [12]:
scoremapViz_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_scoremapViz'
if not os.path.exists(scoremapViz_rootdir):
    os.makedirs(scoremapViz_rootdir)

In [13]:
# labels_to_detect = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']


labels_to_detect = ['5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
                     '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']


In [14]:
downscale_factor = 8

In [ ]:
stack = 'MD585'

first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

In [ ]:
for sec in range(first_detect_sec, last_detect_sec + 1):

    print sec
    
    scoremapViz_dir = os.path.join(scoremapViz_rootdir, stack, '%04d'%sec)
    if not os.path.exists(scoremapViz_dir):
        os.makedirs(scoremapViz_dir)

    dm = DataManager(stack=stack, section=sec)
    dm._load_image(['rgb-jpg'])

    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}

    for l in labels_to_detect:
        print l
        
        scoremap_bp_filepath = scoremaps_rootdir + '/%(stack)s/%(slice)04d/%(stack)s_%(slice)04d_roi1_denseScoreMapLossless_%(label)s.bp' \
            % {'stack': stack, 'slice': sec, 'label': l}
            
        if not os.path.exists(scoremap_bp_filepath):
            sys.stderr.write('No scoremap for %s for section %d\n' % (l, sec))
            continue
    
#         dense_score_map_lossless = bp.unpack_ndarray_file(scoremap_bp_filepath)

        scoremap = bp.unpack_ndarray_file(os.path.join(scoremaps_rootdir, stack, '%04d'%sec,
                                                   '%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s.bp' % \
                                                   {'stack': stack, 'sec': sec, 'label': l}))   
        dataset = stack + '_' + '%04d'%sec + '_roi1'
        
        interpolation_xmin, interpolation_xmax, \
        interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_rootdir, stack, '%04d'%sec,
                                                                         '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                        {'dataset': dataset, 'label': l})).astype(np.int)
        
        dense_score_map_lossless = np.pad(scoremap, ((interpolation_ymin, dm.image_height-interpolation_ymax-1), 
                                      (interpolation_xmin, dm.image_width-interpolation_xmax-1)),
                                  mode='constant', constant_values=0)

#         dense_score_map_lossless = np.load(os.path.join(scoremaps_rootdir, stack, '%04d'%sec,
#                                                         dataset + '_denseScoreMapLossless_%s.npz' % l))['arr_0']
        scoremap_viz = plt.cm.hot(dense_score_map_lossless[::downscale_factor, ::downscale_factor])

        viz = alpha_blending(scoremap_viz[..., :3], dm.image_rgb_jpg[::downscale_factor, ::downscale_factor], .3, 1.)
        
        cv2.putText(viz, l, (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)
        
        cv2.imwrite(os.path.join(scoremapViz_dir, dataset+'_scoremapViz_%s.jpg' % l), 
                    img_as_ubyte(viz[..., [2,1,0]]))
        

In [6]:
def func(sec):
    
#     print sec
    
    global stack
    
    scoremapViz_dir = os.path.join(scoremapViz_rootdir, stack, '%04d'%sec)
    if not os.path.exists(scoremapViz_dir):
        os.makedirs(scoremapViz_dir)

    dm = DataManager(stack=stack, section=sec)
    dm._load_image(['rgb-jpg'])

    dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}

    for l in labels_to_detect:
#         print l
        
        scoremap_bp_filepath = scoremaps_rootdir + '/%(stack)s/%(slice)04d/%(stack)s_%(slice)04d_roi1_denseScoreMapLossless_%(label)s.bp' \
            % {'stack': stack, 'slice': sec, 'label': l}
            
        if not os.path.exists(scoremap_bp_filepath):
#             sys.stderr.write('No scoremap for %s for section %d\n' % (l, sec))
            continue
    
#         dense_score_map_lossless = bp.unpack_ndarray_file(scoremap_bp_filepath)

#         t = time.time()
        scoremap = bp.unpack_ndarray_file(os.path.join(scoremaps_rootdir, stack, '%04d'%sec,
                                                   '%(stack)s_%(sec)04d_roi1_denseScoreMapLossless_%(label)s.bp' % \
                                                   {'stack': stack, 'sec': sec, 'label': l}))   
#         print 1, time.time() - t
        
        dataset = stack + '_' + '%04d'%sec + '_roi1'
        
        interpolation_xmin, interpolation_xmax, \
        interpolation_ymin, interpolation_ymax = np.loadtxt(os.path.join(scoremaps_rootdir, stack, '%04d'%sec,
                                                                         '%(dataset)s_denseScoreMapLossless_%(label)s_interpBox.txt' % \
                                        {'dataset': dataset, 'label': l})).astype(np.int)
        
        
#         t = time.time()
        
        dense_score_map_lossless = np.zeros((dm.image_height, dm.image_width))
        dense_score_map_lossless[interpolation_ymin:interpolation_ymax+1,
                                interpolation_xmin:interpolation_xmax+1] = scoremap.copy()
        
#         dense_score_map_lossless = np.pad(scoremap, ((interpolation_ymin, dm.image_height-interpolation_ymax-1), 
#                                       (interpolation_xmin, dm.image_width-interpolation_xmax-1)),
#                                   mode='constant', constant_values=0)
#         print 2, time.time() - t
    
#         dense_score_map_lossless = np.load(os.path.join(scoremaps_rootdir, stack, '%04d'%sec,
#                                                         dataset + '_denseScoreMapLossless_%s.npz' % l))['arr_0']
        
#         t = time.time()
        scoremap_viz = plt.cm.hot(dense_score_map_lossless[::downscale_factor, ::downscale_factor])
        
#         scoremap_viz = plt.cm.hot(scoremap[::downscale_factor, ::downscale_factor])
#         viz_roi = alpha_blending(scoremap_viz[..., :3],, .3, 1.)

#         print 3, time.time() - t
        
#         t = time.time()
        
    
        viz = (.3 * img_as_ubyte(scoremap_viz[..., :3]) + .7 * dm.image_rgb_jpg[::downscale_factor, ::downscale_factor]).astype(np.uint8)
    
#         viz = alpha_blending(scoremap_viz[..., :3], dm.image_rgb_jpg[::downscale_factor, ::downscale_factor], .3, 1.)

#         print 4, time.time() - t    

        cv2.putText(viz, l, (50, 50), cv2.FONT_HERSHEY_DUPLEX, 2, ((0,0,0)), 3)
        
#         t = time.time()
        cv2.imwrite(os.path.join(scoremapViz_dir, dataset+'_scoremapViz_%s.jpg' % l), 
                    img_as_ubyte(viz[..., [2,1,0]]))
        
#         print 5, time.time() - t

In [7]:
import time

In [8]:
sys.stderr.flush()
sys.stdout.flush()

In [15]:
for stack in ['MD585']:
# for stack in ['MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:

    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]
    
#     for sec in range(first_detect_sec, last_detect_sec):
    for sec in [142]:
        
        print sec
        t = time.time()
    
        _ = func(sec)
#     _ = Parallel(n_jobs=4)(delayed(func)(sec) for sec in range(first_detect_sec, first_detect_sec+2))
#     _ = Parallel(n_jobs=16)(delayed(func)(sec) for sec in range(first_detect_sec, first_detect_sec+16))
    
        print time.time() - t


142
27.2448499203


In [ ]:
for stack in ['MD585']:
# for stack in ['MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:

    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]
    
    t = time.time()
    
    # 391s
    
    _ = Parallel(n_jobs=8)(delayed(func)(sec) for sec in range(first_detect_sec, last_detect_sec))
    
    print time.time() - t